## Summary statistics

In [2]:
from kipoi_cadd.data import cadd_train_valid_data, train_test_split_indexes, CaddDataset
from kipoi_cadd.utils import dump_to_pickle, load_pickle
import pandas as pd
import time
from tqdm import tqdm, trange
import numpy as np
from kipoi.data_utils import numpy_collate_concat
from sklearn.preprocessing import StandardScaler
%load_ext line_profiler

In [3]:
training_dir = "/s/project/kipoi-cadd/data/raw/v1.3/training_data/"
lmdb_dir = training_dir + "lmdb"
csv_file = training_dir + "training_data.imputed.csv"
valid_id_file = training_dir + "valid_idx.pkl"
all_ids_file = training_dir + "variant_ids.pkl"
ids_10k_file = training_dir + "sample_indices/ids_10k.pkl"
first_10k_file = training_dir + "sample_indices/first_10k_ids.pkl"
last_10k_file = training_dir + "sample_indices/last_10k_ids.pkl"
last_10k_csv = training_dir + "last10k.csv"

## Fist 10k rows of the csv file

In [4]:
df10k = pd.read_csv(csv_file, nrows=10000)
df10k_y = [0 if s == -1 else 1 for s in df10k.y]
df10k.drop('y', axis=1, inplace=True)
pd_description = df10k.describe(include='all')
dump_to_pickle(training_dir + "stats/summary_stats_first10k.pkl", pd_description)
pd_description

,RefxA,RefxC,RefxG,RefxT,RefxN,AltxA,AltxC,AltxG,AltxT,AltxN,...,YxM,YxN,YxP,YxQ,YxR,YxS,YxT,YxV,YxW,YxY
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.0,10000.000000,10000.000000,10000.000000,10000.000000,10000.0,...,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.000000
mean,0.221500,0.274200,0.287200,0.217100,0.0,0.256200,0.257600,0.242800,0.243400,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000400
std,0.415277,0.446133,0.452478,0.412292,0.0,0.436555,0.437334,0.428797,0.429156,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.019997
min,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
75%,0.000000,1.000000,1.000000,0.000000,0.0,1.000000,1.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
max,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000


In [5]:
"""
all_ids = load_pickle(all_ids_file)
first_10k = all_ids.iloc[:10000]
first_10k_file = training_dir + "first_10k_ids.pkl"
dump_to_pickle(first_10k_file, first_10k)
"""

'\nall_ids = load_pickle(all_ids_file)\nfirst_10k = all_ids.iloc[:10000]\nfirst_10k_file = training_dir + "first_10k_ids.pkl"\ndump_to_pickle(first_10k_file, first_10k)\n'

In [6]:
first_10k_ds = CaddDataset(lmdb_dir, first_10k_file)
X, y = first_10k_ds.load_all(shuffle=False, drop_last=False)

100%|██████████| 157/157 [00:00<00:00, 187.46it/s]


In [7]:
np.allclose(np.array(df10k, dtype=np.float16), X, rtol=1e-05, atol=1e-07)

True

In [8]:
np.allclose(np.array(df10k_y, dtype=np.float16), y, rtol=1e-05, atol=1e-07)

True

## Last 10k rows of the csv file

In [9]:
"""
! head {csv_file} -n 1 > last10k.csv
! tail {csv_file} -n 10000 >> last10k.csv
"""

'\n! head {csv_file} -n 1 > last10k.csv\n! tail {csv_file} -n 10000 >> last10k.csv\n'

In [10]:
df_last10k = pd.read_csv(last_10k_csv)
df_last10k_y = [0 if s == -1 else 1 for s in df_last10k.y]
df_last10k.drop('y', axis=1, inplace=True)
pd_description_last10k = df_last10k.describe(include='all')
dump_to_pickle(training_dir + "stats/summary_stats_last10k.pkl", pd_description_last10k)
pd_description_last10k

,RefxA,RefxC,RefxG,RefxT,RefxN,AltxA,AltxC,AltxG,AltxT,AltxN,...,YxM,YxN,YxP,YxQ,YxR,YxS,YxT,YxV,YxW,YxY
count,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,...,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0
mean,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
"""
all_ids = load_pickle(all_ids_file)
last_10k = all_ids.tail(10000)
dump_to_pickle(last_10k_file, last_10k)
"""
last_10k = load_pickle(last_10k_file)

In [12]:
last_10k_ds = CaddDataset(lmdb_dir, last_10k_file)
X, y = last_10k_ds.load_all(shuffle=False, drop_last=False)

100%|██████████| 157/157 [00:00<00:00, 294.97it/s]


In [13]:
ds = CaddDataset(lmdb_dir, last_10k_file)
full_info = ds.load_all_with_metadata(shuffle=False, drop_last=False)

100%|██████████| 157/157 [00:00<00:00, 359.19it/s]


In [14]:
equality = np.array(df_last10k, dtype=np.float16) == X

In [15]:
dif_idx = {}
for i in range(10000):
    for j in range(1063):
        if not equality[i,j]:
            if i in dif_idx.keys():
                dif_idx[i].append(j)
            else:
                dif_idx[i] = []

In [16]:
len([v if "N" in v else "" for v in full_info['metadata']['variant_id']])

10000

In [17]:
for k, v in dif_idx.items():
    print(len(v))

26
26
29


In [19]:
# dif_idx has only these rows 915, 7008, 9223
pd.DataFrame([np.array(df_last10k, dtype=np.float16)[915], full_info['inputs'][915]]).iloc[:,10:50]

,10,11,12,13,14,15,16,17,18,19,...,40,41,42,43,44,45,46,47,48,49
0,0.0,1.0,0.0,8.0,0.5,0.0,0.0,0.0,0.0,1.0,...,0.117981,0.117981,0.0,0.0,800.0,-11.0,0.280029,-0.280029,-2.929688,-2.279297
1,0.0,0.0,1.0,2.0,0.5,0.0,0.0,0.0,0.0,1.0,...,0.117981,0.117981,0.0,0.0,800.0,-59.0,4.949219,-0.160034,-4.671875,-2.210938


In [20]:
full_info['inputs'][915]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float16)

In [22]:
np.allclose(np.array(df_last10k, dtype=np.float16), full_info['inputs'], rtol=1e-05, atol=1e-07)

False

In [21]:
np.allclose(np.array(df_last10k_y, dtype=np.float16), full_info['targets'], rtol=1e-05, atol=1e-07)

True

## Standardize dataset using StandardScaler

In [23]:
first_10k_ds = CaddDataset(lmdb_dir, first_10k_file)
X, y = first_10k_ds.load_all(shuffle=False, drop_last=False)

100%|██████████| 157/157 [00:00<00:00, 485.84it/s]


In [24]:
scaler = StandardScaler(with_mean=False) # Used with_mean=False, for it to work with sparse matrices too.
X = X.astype(np.float32)
X_transformed = scaler.fit_transform(X)

In [25]:
pd.DataFrame(X).describe(include='all')

,0,1,2,3,4,5,6,7,8,9,...,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.0,10000.000000,10000.000000,10000.000000,10000.000000,10000.0,...,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.000000
mean,0.221500,0.274200,0.287200,0.217100,0.0,0.256200,0.257600,0.242800,0.243400,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000400
std,0.415277,0.446133,0.452478,0.412292,0.0,0.436555,0.437334,0.428797,0.429156,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.019997
min,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
75%,0.000000,1.000000,1.000000,0.000000,0.0,1.000000,1.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
max,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000


In [19]:
pd.DataFrame(X_transformed).describe(include='all').loc[:,124]

count    1.000000e+04
mean    -3.051758e-09
std      1.000055e+00
min     -9.438202e-02
25%     -9.438202e-02
50%     -9.438202e-02
75%     -9.438202e-02
max      3.436163e+01
Name: 124, dtype: float64

In [30]:
X_small = X_transformed.astype(np.float16)
pd.DataFrame(X_small).describe(include='all').loc[:,124]

count    10000.000000
mean         0.000028
std          1.000000
min         -0.094360
25%         -0.094360
50%         -0.094360
75%         -0.094360
max         34.375000
Name: 124, dtype: float64

In [26]:
# Save scaler for later
dump_to_pickle(training_dir + "stats/standard_scaler_first10k.pkl", scaler)

## Standardize dataset using MinMaxScaler

In [27]:
first_10k_ds = CaddDataset(lmdb_dir, first_10k_file)
X, y = first_10k_ds.load_all(shuffle=False, drop_last=False)

100%|██████████| 157/157 [00:00<00:00, 355.74it/s]


In [28]:
scaler = StandardScaler(with_mean=False) # Used with_mean=False, for it to work with sparse matrices too.
X = X.astype(np.float32)
X_transformed = scaler.fit_transform(X)
print(scaler.mean_)

[0.2215 0.2742 0.2872 ... 0.     0.     0.0004]


In [29]:
# Save scaler for later
dump_to_pickle(training_dir + "stats/minmax_scaler_first10k.pkl", scaler)

In [40]:
recovered = load_pickle(training_dir + "stats/minmax_scaler_first10k.pkl")
recovered.transform(np.ones_like(X))

array([[ 1.8746974,  1.626955 ,  1.5753938, ...,  1.       ,  1.       ,
        49.986115 ],
       [ 1.8746974,  1.626955 ,  1.5753938, ...,  1.       ,  1.       ,
        49.986115 ],
       [ 1.8746974,  1.626955 ,  1.5753938, ...,  1.       ,  1.       ,
        49.986115 ],
       ...,
       [ 1.8746974,  1.626955 ,  1.5753938, ...,  1.       ,  1.       ,
        49.986115 ],
       [ 1.8746974,  1.626955 ,  1.5753938, ...,  1.       ,  1.       ,
        49.986115 ],
       [ 1.8746974,  1.626955 ,  1.5753938, ...,  1.       ,  1.       ,
        49.986115 ]], dtype=float32)

In [42]:
recovered.mean_

array([0.2215, 0.2742, 0.2872, ..., 0.    , 0.    , 0.0004])